In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install opencv-python==4.1.2.30

In [ ]:
import cv2
import os

In [ ]:
cv2.__version__

In [ ]:
image_list = []
i = 0
for image in os.listdir('../input/celeba-dataset/img_align_celeba/img_align_celeba'):
    image_list.append(cv2.resize(cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/'+image, 0), (256,256)))
    if i  == 10000:
        break
    i += 1

In [ ]:
len(image_list)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(image_list[18], cmap='gray')

In [ ]:
import numpy as np
image_list = np.array(image_list)
image_list = (image_list - 127.5) / 127.5

In [ ]:
train_image = image_list.reshape(image_list.shape[0], 256, 256, 1).astype('float32')
del image_list

In [ ]:
import tensorflow as tf
BUFFER_SIZE = 10001
BATCH_SIZE = 64
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_image).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
del train_image

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(64*64*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((64, 64, 256)))
    assert model.output_shape == (None, 64, 64, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 64, 64, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 128, 128, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 256, 256, 1)

    return model

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import tensorflow as tf

from IPython import display

In [ ]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')


In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                                     input_shape=[256, 256, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


In [ ]:

discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)




In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)



In [ ]:

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)



In [ ]:

checkpoint_dir = '/checkpoint'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)


EPOCHS = 1000
noise_dim = 100
num_examples_to_generate = 16

# We will reuse this seed overtime (so it's easier)
# to visualize progress in the animated GIF)
seed = tf.random.normal([num_examples_to_generate, noise_dim])



In [ ]:

# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
# @tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=True)
      fake_output = discriminator(generated_images, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)




In [ ]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(24,24))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('./images/image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()



In [ ]:
!mkdir images
!mkdir checkpoint

In [ ]:
train(train_dataset, EPOCHS)

In [ ]:
import requests

def get_predictions(path):
   url = "https://referral-mysql.herokuapp.com/api/uploadImage/singleImage"
   payload = {"userImage": (path, open(path, "rb"), "image/jpg")}

   headers = {'content-type': "multipart/form-data"}

   response = requests.post(url, data=payload, headers=headers, stream=True)

   print(response.text)

In [ ]:
get_predictions('../input/celeba-dataset/img_align_celeba/img_align_celeba/000001.jpg')


In [ ]:
import cv2

In [ ]:
a = cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/000001.jpg',0)

In [ ]:
import requests
url = 'https://referral-mysql.herokuapp.com/api/uploadImage/singleImage'
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
}
path = '../input/celeba-dataset/img_align_celeba/img_align_celeba/000001.jpg'
files = {
        'userImage': ('000001.jpg', open(path, 'rb')),
    }
req = requests.post(url, files=files)

In [ ]:
str(req)

In [ ]:
url = "https://{domain_prefix}.vendhq.com/api/2.0/products/{product_id}/actions/image_upload"
files = {'image': open('{file_path}', 'rb')}
headers = {
    'authorization': "Bearer {token}"
}
response = requests.request("POST", url, files=files, headers=headers)

print(response.text)